In [1]:
import os
import json
import numpy as np 
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer

/Users/albadiegovelarde/Library/CloudStorage/OneDrive-Repsol/Proyectos/student_marks/students_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("student_grades.json", "r") as f:
    student_grades = json.load(f)

In [4]:

embedder = SentenceTransformer("all-MiniLM-L6-v2")

student_vectors = {}
for student_id, skills in student_grades.items():
    grades = []
    explanations = []

    for skill, data in skills.items():
        if skill == "global grade":
            continue
        grades.append(data["grade"])
        explanations.append(data["explanation"])

    # Mean embedding of the explanations
    explanation_embeddings = embedder.encode(explanations)
    explanation_vector = np.mean(explanation_embeddings, axis=0)

    # Concatenate grades and embeddings
    full_vector = np.concatenate([grades, explanation_vector])
    student_vectors[student_id] = full_vector

### Clustering
X = np.array(list(student_vectors.values()))
student_ids = list(student_vectors.keys())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df_clusters = pd.DataFrame({
    "student_id": student_ids,
    "cluster": clusters
})

print(df_clusters)


  student_id  cluster
0  student_1        0
1  student_2        0
2  student_3        1
3  student_4        1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from sklearn.decomposition import PCA
import plotly.express as px

## PCA for visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

df_clusters = pd.DataFrame({
    "student_id": student_ids,
    "cluster": clusters,
    "PC1": X_pca[:, 0],
    "PC2": X_pca[:, 1]
})

df_clusters["cluster"] = df_clusters["cluster"].astype(str)

fig = px.scatter(
    df_clusters,
    x="PC1",
    y="PC2",
    color="cluster",   # ahora es categórico
    text="student_id",
    title="Student clustering with KMeans (PCA)"
)

# Añadir centroides
centroids_pca = pca.transform(kmeans.cluster_centers_)
fig.add_scatter(
    x=centroids_pca[:, 0],
    y=centroids_pca[:, 1],
    mode="markers",
    marker=dict(color="black", size=15, symbol="x"),
    name="Centroids"
)

fig.show()
